<a href="https://colab.research.google.com/github/IqmanS/Machine-Learning-Notebooks/blob/main/parallel_programming/Parallel_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import time
import multiprocessing #older way
import concurrent.futures
import threading
import logging
import os
import numpy as np

In [17]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

CPU = os.cpu_count()

DEBUG:root:test


# Serial

In [13]:
start = time.perf_counter()

def do_something():
    print("Sleep Start")
    time.sleep(1)
    print("Sleep End")

do_something()
do_something()

finish = time.perf_counter()
print("Time Taken:",round(finish-start,2))

Sleep Start
Sleep End
Sleep Start
Sleep End
Time Taken: 2.0


# Multiprocessing (older way)

In [14]:
start = time.perf_counter()

p1 = multiprocessing.Process(target = do_something,args=())
p2 = multiprocessing.Process(target = do_something,args=())

p1.start()
p2.start()

p1.join()
p2.join()

finish = time.perf_counter()
print("Time Taken:",round(finish-start,2))

Sleep Start
Sleep Start
Sleep End
Sleep End
Time Taken: 1.05


In [16]:
start = time.perf_counter()

process = []
for _ in range(10):
    p = multiprocessing.Process(target = do_something,args=())
    p.start()
    process.append(p)

for p in process:
    p.join()

finish = time.perf_counter()
print("Time Taken:",round(finish-start,2))

Sleep Start
Sleep StartSleep Start

Sleep StartSleep Start

Sleep Start
Sleep Start
Sleep Start
Sleep Start
Sleep Start
Sleep End
Sleep End
Sleep End
Sleep EndSleep End

Sleep EndSleep End

Sleep End
Sleep End
Sleep End
Time Taken: 1.18


# concurrent.futures.ProcessPoolExecutor and ThreadPoolExecutor (New)


* ProcessPool is for CPU bound tasks so you can benefit from multiple CPU.
* ThreadPool is for IO bound tasks so you can benefit from IO wait.




In [37]:
def do_something(seconds):
    print(f"Sleep Start for {seconds}s")
    time.sleep(seconds)
    return f"Sleep End of {seconds}s"

In [38]:
# ProcessPoolExecutor
start = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
    # f1 = executor.submit(do_something,1)
    results = [executor.submit(do_something,1) for _ in range(10)]

    for f in concurrent.futures.as_completed(results):
        print(f.result())


finish = time.perf_counter()
print("Time Taken:",round(finish-start,2))

Sleep Start for 1sSleep Start for 1s

Sleep Start for 1s
Sleep Start for 1s
Sleep End of 1s
Sleep End of 1s
Sleep Start for 1s

Sleep Start for 1sSleep End of 1s
Sleep End of 1s
Sleep Start for 1s
Sleep Start for 1s
Sleep End of 1s
Sleep End of 1s
Sleep Start for 1s
Sleep Start for 1s
Sleep End of 1s
Sleep End of 1s
Sleep End of 1s
Sleep End of 1s
Time Taken: 5.1


In [39]:
# ThreadPoolExecutor
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    # f1 = executor.submit(do_something,1)
    results = [executor.submit(do_something,1) for _ in range(10)]

    for f in concurrent.futures.as_completed(results):
        print(f.result())


finish = time.perf_counter()
print("Time Taken:",round(finish-start,2))

Sleep Start for 1s
Sleep Start for 1s
Sleep Start for 1s
Sleep Start for 1s
Sleep Start for 1s
Sleep Start for 1s
Sleep Start for 1sSleep End of 1s

Sleep Start for 1s
Sleep End of 1s
Sleep Start for 1sSleep End of 1s

Sleep Start for 1sSleep End of 1s

Sleep End of 1s
Sleep End of 1s
Sleep End of 1s
Sleep End of 1s
Sleep End of 1s
Sleep End of 1s
Time Taken: 2.01


In [40]:
# ProcessPoolExecutor
start = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
    # f1 = executor.submit(do_something,1)
    results = [executor.submit(do_something,_) for _ in range(5,0,-1)]

    for f in concurrent.futures.as_completed(results):
        print(f.result())


finish = time.perf_counter()
print("Time Taken using ProcessPoolExecutor:",round(finish-start,2))

print("\n","--"*50)

# ThreadPoolExecutor
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    # f1 = executor.submit(do_something,1)
    results = [executor.submit(do_something,_) for _ in range(5,0,-1)]

    for f in concurrent.futures.as_completed(results):
        print(f.result())


finish = time.perf_counter()
print("Time Taken using ThreadPoolExecutor:",round(finish-start,2))

Sleep Start for 4sSleep Start for 5s

Sleep Start for 3s
Sleep End of 4s
Sleep Start for 2s
Sleep End of 5s
Sleep Start for 1s
Sleep End of 3s
Sleep End of 2s
Sleep End of 1s
Time Taken using ProcessPoolExecutor: 8.06

 ----------------------------------------------------------------------------------------------------
Sleep Start for 5s
Sleep Start for 4s
Sleep Start for 3sSleep Start for 2s
Sleep Start for 1s

Sleep End of 1s
Sleep End of 2s
Sleep End of 3s
Sleep End of 4s
Sleep End of 5s
Time Taken using ThreadPoolExecutor: 5.01


# Using 'map'

In [43]:
# ProcessPoolExecutor
start = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
    results = executor.map(do_something,list(range(5,0,-1)))

    for f in results:
        print(f)


finish = time.perf_counter()
print("Time Taken using ProcessPoolExecutor:",round(finish-start,2))

print("\n","--"*50)

# ThreadPoolExecutor
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(do_something,list(range(5,0,-1)))

    for f in results:
        # Handle exceptions here
        print(f)


finish = time.perf_counter()
print("Time Taken using ThreadPoolExecutor:",round(finish-start,2))

Sleep Start for 5s
Sleep Start for 4s
Sleep Start for 3s
Sleep Start for 2s
Sleep End of 5s
Sleep End of 4s
Sleep Start for 1s
Sleep End of 3s
Sleep End of 2s
Sleep End of 1s
Time Taken using ProcessPoolExecutor: 8.06

 ----------------------------------------------------------------------------------------------------
Sleep Start for 5s
Sleep Start for 4s
Sleep Start for 3s
Sleep Start for 2s
Sleep Start for 1s
Sleep End of 5s
Sleep End of 4s
Sleep End of 3s
Sleep End of 2s
Sleep End of 1s
Time Taken using ThreadPoolExecutor: 5.0


# Creating 10000 (500x500) matrices and multiplying them

In [74]:
def CreateAndMultiply(): # Creates 2 Matrices and multiples them
    mat1 = 10 * np.random.random_sample((500,500))
    mat2 = 10 * np.random.random_sample((500,500))
    mul = mat1*mat2

In [72]:
# Using ProcessPoolExecutor
start = time.perf_counter()

with concurrent.futures.ProcessPoolExecutor() as executor:
    [executor.submit(CreateAndMultiply) for _ in range(5000)]

finish = time.perf_counter()
print("Time Taken using ProcessPoolExecutor:",round(finish-start,2))


Time Taken using ProcessPoolExecutor: 38.63


In [73]:
# Using ThreadPoolExecutor
start = time.perf_counter()

with concurrent.futures.ThreadPoolExecutor() as executor:
    [executor.submit(CreateAndMultiply) for _ in range(5000)]


finish = time.perf_counter()
print("Time Taken using ThreadPoolExecutor:",round(finish-start,2))


Time Taken using ThreadPoolExecutor: 40.94
